In [ ]:
%pip install pwntools
%pip install sympy

In [ ]:
from pwn import remote
from Crypto.Util.number import inverse, isPrime
from sympy import primerange
from sympy.ntheory.modular import crt
import json
from functools import reduce
import random

In [ ]:
def json_recv(r):
    line = r.recvline()
    return json.loads(line.decode())

def json_send(r, hsh):
    request = json.dumps(hsh).encode()
    r.sendline(request)

In [ ]:
def generate_basis(n):
    basis = [True] * n
    for i in range(3, int(n**0.5)+1, 2):
        if basis[i]:
            basis[i*i::2*i] = [False]*((n-i*i-1)//(2*i)+1)
    return [2] + [i for i in range(3, n, 2) if basis[i]]

def miller_rabin(n, b):
    """
    Miller Rabin test testing over all
    prime basis < b
    """
    basis = generate_basis(b)
    if n == 2 or n == 3:
        return True

    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for b in basis:
        x = pow(b, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

As we are using miller-rabin primality test, we need to remember that this algorithm has some false positives, depending on the chosen base. If we'll use a specific number which pass this test for the calculated base but is not prime, we can get all the characters we want from the server.

Luckily, this paper provides an algorithm to generate such primes for a given base:

Arnault, F. (1995). Constructing Carmichael numbers which are strong pseudoprimes to several bases. Journal of Symbolic Computation, 20(2), 151-161.

In [ ]:
#based on https://www.sciencedirect.com/science/article/pii/S074771718571042
random.seed(12345)

primes = list(primerange(3, 5000)) #list of "small" primes

k2, k3 = primes[40], primes[35]
k_lst = [1, k2, k3]
bases = generate_basis(64)

bases_4 = [4*b for b in bases]

Sb = {b : {p%(4*b) for p in primes if pow(b, (p-1)//2, p)==p-1} for b in bases}
Sb_k = { (b, k) : {(inverse(k, 4*b)*(s+k-1))%(4*b) for s in Sb[b]} for b in bases for k in k_lst}

p1_mods = {b : reduce(set.intersection, [Sb_k[(b, k)] for k in k_lst]) for b in bases}

psudo_prime = None
while psudo_prime is None:
    remainders = [random.sample(list(x), 1)[0] for x in p1_mods.values()]
    a = bases_4 + k_lst[1:]
    n = (*remainders, inverse(-k3, k2), inverse(-k2, k3))
    sol = crt(a, n)
    if sol is None: continue

    p1_candidate, n_candidate = sol
    for i in range(1000):
        p1 = p1_candidate + (10**40+i)*n_candidate # the factor 10**40 is to make sure psudo_prime is 600-900 bits
        p2, p3 = k2*(p1-1)+1, k3*(p1-1)+1
        if isPrime(p1) and isPrime(p2) and isPrime(p3):
            p_product = p1*p2*p3
            if p_product.bit_length() > 600 or p_product.bit_length() < 900:
                psudo_prime = p_product
                break

print(p1, p2, p3, psudo_prime)

In [ ]:
print("p1 is prime:", isPrime(p1))
print("p2 is prime:", isPrime(p2))
print("p3 is prime:", isPrime(p3))
print("psudo_prime is prime:", isPrime(psudo_prime))
print("psudo_prime passes rabin miller test:", miller_rabin(psudo_prime, 64))

In [ ]:
HOST = "socket.cryptohack.org"
PORT = 13385

r = remote(HOST, PORT)
line = r.recvline()
line

In [ ]:
json_send(r, {"prime": psudo_prime, "base": p1})
line = r.recvline()
line